<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#BERT" data-toc-modified-id="BERT-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [5]:
#!pip install torch
#!pip install transformers

     |████████████████████████████████| 3.5 MB 409 kB/s eta 0:00:01
     |████████████████████████████████| 67 kB 6.9 MB/s  eta 0:00:01
     |████████████████████████████████| 3.6 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 3.2 MB/s eta 0:00:01


In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

### BERT

Векторизуем исходные тексты с помощью трансформации BERT.

In [2]:
df = pd.read_csv('/Users/vasily/Learning/Data Analytics/Projects/Text classification/toxic_comments.csv')

In [3]:
df[df['toxic']==1]

,text,toxic
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,"Bye! \n\nDon't look, come or think of comming ...",1
42,You are gay or antisemmitian? \n\nArchangel WH...,1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
...,...,...
159494,"""\n\n our previous conversation \n\nyou fuckin...",1
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1
159541,Your absurd edits \n\nYour absurd edits on gre...,1
159546,"""\n\nHey listen don't you ever!!!! Delete my e...",1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [33]:
df_bert = df.sample(n=1000, random_state=123)

text     16225
toxic    16225
dtype: int64

In [15]:
df_bert.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 137651 to 19345
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   toxic   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 23.4+ KB


In [16]:
#Tokenizing words
tokenizer = transformers.BertTokenizer(
    vocab_file='/Users/vasily/Learning/Data Analytics/Projects/Text classification/vocab.txt')

"""
tokenized = df_bert['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = max(map(len, tokenized))

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
"""

"\ntokenized = df_bert['text'].apply(\n    lambda x: tokenizer.encode(x, add_special_tokens=True))\n\nmax_len = max(map(len, tokenized))\n\npadded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])\n\nattention_mask = np.where(padded != 0, 1, 0)\n"

In [17]:
#КОД РЕВЬЮЕРА
from torch.nn.utils.rnn import pad_sequence
max_seq_length = 512
tokenized = df_bert['text'].apply((lambda x: tokenizer.encode(x, 
                                                        truncation=True, 
                                                        max_length=max_seq_length, 
                                                        add_special_tokens=True)))

padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True)
attention_mask = np.where(padded != 0, 1, 0)

In [18]:
config = transformers.BertConfig.from_json_file(
    '/Users/vasily/Learning/Data Analytics/Projects/Text classification/bert_config.json')
#model = transformers.BertModel.from_pretrained('bert-base-uncased', config=config)
model = transformers.BertModel.from_pretrained('/Users/vasily/Learning/Data Analytics/Projects/Text classification/pytorch_model.bin', config=config)

Some weights of the model checkpoint at /Users/vasily/Learning/Data Analytics/Projects/Text classification/pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
padded

tensor([[  101, 18257,   214,  ...,     0,     0,     0],
        [  101,   107,  7163,  ...,     0,     0,     0],
        [  101,  7163, 17469,  ...,     0,     0,     0],
        ...,
        [  101,   107,   646,  ...,     0,     0,     0],
        [  101,   107,  1436,  ...,     0,     0,     0],
        [  101,  1112,  2395,  ...,     0,     0,     0]])

In [19]:
padded.shape
#padded = padded[:, 0:512]

torch.Size([1000, 512])

In [141]:
#attention_mask = attention_mask[:, 0:512]

In [20]:
attention_mask.shape

(1000, 512)

In [21]:
%%time

batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 24min 54s, sys: 9min 11s, total: 34min 5s
Wall time: 27min 15s


In [22]:
features = np.concatenate(embeddings)

In [23]:
features.shape

(1000, 768)

## Обучение

### Logistic Regression

Обучим и применим модель логистической регрессии на векторизованных с помощью BERT текстах.

In [79]:
#Prepare train and test data
features_train, features_test = train_test_split(features, test_size=0.5, random_state=12345)
target_train, target_test = train_test_split(df_bert['toxic'], test_size=0.5, random_state=12345)

In [30]:
#Train Logistic Regression model
model = LogisticRegression(solver='liblinear')
model.fit(features_train, target_train)

LogisticRegression(solver='liblinear')

In [31]:
predictions = model.predict(features_test)

In [32]:
print('F1 score with BERT and Logistic Regression:',f1_score(target_test, predictions))

F1 score with BERT and Logistic Regression: 0.8624229979466119


### CatBoost

Модель Catboost может сама выполнять трансформацию текстов в признаки. Предварительной векторизации не требуется.

In [185]:
#Prepare train and test data
features_catboost=df.drop(['toxic'], axis=1)
target_catboost=df['toxic']

In [187]:
features_train, features_test, target_train, target_test = train_test_split(features_catboost, target_catboost, test_size=0.25, random_state=12345)

In [188]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119678 entries, 111565 to 77285
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    119678 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB


In [207]:
parameters = {
    'tokenizers': [
        {
            'tokenizer_id': 'Sense',
            'separator_type': 'BySense',
            'lowercasing': 'True',
            'token_types':['Word', 'Number', 'SentenceBreak'],
            'sub_tokens_policy':'SeveralTokens'
        }      
    ],
    'dictionaries': [
        {
            'dictionary_id': 'Word',
            'max_dictionary_size': '50000'
        }
    ],
    'feature_calcers': [
        'BoW:top_tokens_count=10000'
    ]}
catboost = CatBoostClassifier(text_features=['text'], iterations=3000, eval_metric='F1', learning_rate=0.05, **parameters)

In [200]:
catboost.fit(features_train, target_train, verbose = 100)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.3679403	total: 233ms	remaining: 11m 39s
100:	learn: 0.5852458	total: 24.9s	remaining: 11m 55s
200:	learn: 0.6492027	total: 50s	remaining: 11m 36s
300:	learn: 0.6886778	total: 1m 14s	remaining: 11m 10s
400:	learn: 0.7135307	total: 1m 40s	remaining: 10m 51s
500:	learn: 0.7318255	total: 2m 8s	remaining: 10m 40s
600:	learn: 0.7477845	total: 2m 36s	remaining: 10m 24s
700:	learn: 0.7576341	total: 3m 4s	remaining: 10m 5s
800:	learn: 0.7671730	total: 3m 32s	remaining: 9m 44s
900:	learn: 0.7743035	total: 4m	remaining: 9m 20s
1000:	learn: 0.7823646	total: 4m 27s	remaining: 8m 54s
1100:	learn: 0.7891621	total: 4m 53s	remaining: 8m 26s
1200:	learn: 0.7945376	total: 5m 20s	remaining: 7m 59s
1300:	learn: 0.7992744	total: 5m 46s	remaining: 7m 31s
1400:	learn: 0.8038432	total: 6m 12s	remaining: 7m 5s
1500:	learn: 0.8075518	total: 6m 39s	remaining: 6m 39s
1600:	learn: 0.8127452	total: 7m 7s	remaining: 6m 13s
1700:	learn: 0.8157758	total: 7m 35s	remaining: 5m 47s
1800:	learn: 0.8188665	total

In [201]:
predictions = catboost.predict(features_test)

In [202]:
predictions.shape

(39893,)

In [204]:
print('F1 score with Catboost:', f1_score(target_test, predictions))

F1 score with Catboost: 0.773996388387276


## Выводы

Наибольшее значение метрики **F1 = 0.77** и самое быстрое обучение показала модель **Catboost**. С помощью же BERT удалось достигнуть лишь F1 = 0.56. Могла сыграть роль малая выборка (чтобы обеспечить разумное время трансформации, обучение проводили всего лишь на 300 текстах, и ограничение максимальной длины тензора предобученной модели в 512).